
<br>
=============================<br>
Fundamental matrix estimation<br>
=============================<br>
This example demonstrates how to robustly estimate <br>
`epipolar geometry <https://en.wikipedia.org/wiki/Epipolar_geometry>` <br>
(the geometry of stereo vision) between two views using sparse ORB feature<br>
correspondences.<br>
The `fundamental matrix <https://en.wikipedia.org/wiki/Fundamental_matrix_(computer_vision)>`_ <br>
relates corresponding points between a pair of<br>
uncalibrated images. The matrix transforms homogeneous image points in one image<br>
to epipolar lines in the other image.<br>
Uncalibrated means that the intrinsic calibration (focal lengths, pixel skew,<br>
principal point) of the two cameras is not known. The fundamental matrix thus<br>
enables projective 3D reconstruction of the captured scene. If the calibration<br>
is known, estimating the essential matrix enables metric 3D reconstruction of<br>
the captured scene.<br>


In [ ]:
import numpy as np
from skimage import data
from skimage.color import rgb2gray
from skimage.feature import match_descriptors, ORB, plot_matches
from skimage.measure import ransac
from skimage.transform import FundamentalMatrixTransform
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(0)

In [ ]:
img_left, img_right, groundtruth_disp = data.stereo_motorcycle()
img_left, img_right = map(rgb2gray, (img_left, img_right))

Find sparse feature correspondences between left and right image.

In [ ]:
descriptor_extractor = ORB()

In [ ]:
descriptor_extractor.detect_and_extract(img_left)
keypoints_left = descriptor_extractor.keypoints
descriptors_left = descriptor_extractor.descriptors

In [ ]:
descriptor_extractor.detect_and_extract(img_right)
keypoints_right = descriptor_extractor.keypoints
descriptors_right = descriptor_extractor.descriptors

In [ ]:
matches = match_descriptors(descriptors_left, descriptors_right,
                            cross_check=True)

Estimate the epipolar geometry between the left and right image.

In [ ]:
model, inliers = ransac((keypoints_left[matches[:, 0]],
                         keypoints_right[matches[:, 1]]),
                        FundamentalMatrixTransform, min_samples=8,
                        residual_threshold=1, max_trials=5000)

In [ ]:
inlier_keypoints_left = keypoints_left[matches[inliers, 0]]
inlier_keypoints_right = keypoints_right[matches[inliers, 1]]

In [ ]:
print(f"Number of matches: {matches.shape[0]}")
print(f"Number of inliers: {inliers.sum()}")

Compare estimated sparse disparities to the dense ground-truth disparities.

In [ ]:
disp = inlier_keypoints_left[:, 1] - inlier_keypoints_right[:, 1]
disp_coords = np.round(inlier_keypoints_left).astype(np.int64)
disp_idxs = np.ravel_multi_index(disp_coords.T, groundtruth_disp.shape)
disp_error = np.abs(groundtruth_disp.ravel()[disp_idxs] - disp)
disp_error = disp_error[np.isfinite(disp_error)]

Visualize the results.

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1)

In [ ]:
plt.gray()

In [ ]:
plot_matches(ax[0], img_left, img_right, keypoints_left, keypoints_right,
             matches[inliers], only_matches=True)
ax[0].axis("off")
ax[0].set_title("Inlier correspondences")

In [ ]:
ax[1].hist(disp_error)
ax[1].set_title("Histogram of disparity errors")

In [ ]:
plt.show()